## Install



In [1]:
!pip -q install pypdf sentence-transformers
!pip -q install jq  tiktoken
!pip -q install langchain langchain_community langchain-qdrant
!pip -q install fastembed
!pip -q install groq
!pip -q install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 100.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 746.6/746.6 kB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
from groq import Groq
from google.colab import userdata



client = Groq(
    api_key=userdata.get('GROQ_API'),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "แนะนำวิธีทำข้าวแกงกระหรี่",
        }
    ],
    model= "meta-llama/llama-4-maverick-17b-128e-instruct" ,  #"llama-3.1-70b-versatile"
)

print(chat_completion.choices[0].message.content)

1.  วัตถุดิบ

*   ข้าวสวย
*   เนื้อสัตว์หรือผักตามชอบ (เช่น เนื้อไก่, เนื้อหมู, หรือเห็ด, แครอท)
*   หอมหัวใหญ่
*   มันฝรั่ง
*   แครอท
*   กระเทียม
*   ซอสแกงกระหรี่ญี่ปุ่น
*   น้ำ
*   น้ำมันพืช
*   น้ำมันงา
*   เกลือ
*   พริกไทย
2.  วิธีทำ

1.  ตั้งกระทะบนไฟกลาง ใส่น้ำมันพืชลงไปพอน้ำมันร้อน ใส่กระเทียมสับลงไปผัดจนหอม
2.  ใส่หอมหัวใหญ่หั่นเต๋าลงไปผัดจนหอมหัวใหญ่เริ่มนิ่ม
3.  เติมมันฝรั่งหั่นเต๋า และแครอทหั่นเต๋าลงไปผัดจนผักเริ่มนิ่ม
4.  ใส่เนื้อสัตว์หรือผักที่คุณชอบลงไปผัดจนสุก
5.  ใส่ซอสแกงกระหรี่ญี่ปุ่นลงไปคนให้เข้ากัน
6.  เติมน้ำลงไปพอท่วมผักและเนื้อสัตว์ แล้วเคี่ยวด้วยไฟอ่อนประมาณ 15-20 นาที จนแกงมีลักษณะข้นและผักสุก
7.  ปรุงรสด้วยเกลือและพริกไทยตามชอบ
8.  ปิดไฟและเติมน้ำมันงาเล็กน้อยเพื่อเพิ่มกลิ่นหอม
9.  ตักข้าวสวยใส่จาน แล้วราดแกงกระหรี่ลงไป เสิร์ฟพร้อมกับผักดองหรือผักสด เช่น กะหล่ำปลีซอย หรือต้นหอมญี่ปุ่นหั่นฝอย

แกงกระหรี่ญี่ปุ่นเป็นอาหารที่นิยมมากในญี่ปุ่น สามารถปรับสูตรได้ตามความชอบส่วนตัว ไม่ว่าจะเป็นเนื้อสัตว์หรือผักที่ใช้ นอกจากนี้การใช้ซอสแกงกระหรี่ญี่ปุ่นทำให้การทำแกงกร

In [3]:
!gdown 1X80o1hlkf6bqpXW-MufBXmatAouVHlTz

Downloading...
From: https://drive.google.com/uc?id=1X80o1hlkf6bqpXW-MufBXmatAouVHlTz
To: /content/สรุปผลงานสภาผู้แทนราษฎร_ชุดที่_26_ปีที่_1_ครั้งที่หนึ่ง.pdf
100% 2.68M/2.68M [00:00<00:00, 91.0MB/s]


In [4]:
import fitz
import json
import os
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import JSONLoader
from langchain.text_splitter import CharacterTextSplitter
from tqdm.auto import tqdm, trange

In [5]:
def process_text(text):
    return text.replace(' า','ำ')

def metadata_func(record: dict, metadata: dict) -> dict:

    metadata["file"] = record["file"]
    metadata["page"] = record["page"]
    metadata['source'] = '-'

    return metadata

In [6]:
## convert pdf to json
pdf_file = '/content/สรุปผลงานสภาผู้แทนราษฎร_ชุดที่_26_ปีที่_1_ครั้งที่หนึ่ง.pdf'

document = fitz.open(pdf_file)
data_json = []
# Extract text from each page
for page_num in range(len(document)):
    page = document.load_page(page_num)
    text = page.get_text("text")
    data_json.append({'page':page_num,
                    'text': process_text(text),
                    'file' : pdf_file[1:]
                    })

output_file_path = "temp_pdf.json"
with open(output_file_path, "w", encoding="utf-8") as json_file:
    json.dump({'pdf': data_json}, json_file, ensure_ascii=False, indent=4)

In [7]:
loader = JSONLoader(
                file_path=output_file_path,
                jq_schema='."pdf"[]',
                content_key="text",
                metadata_func=metadata_func,
                text_content=False
                )

pages = loader.load()
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=1500,
                                                            chunk_overlap=0,
                                                            separator = "\n\n")
docs = text_splitter.split_documents(pages)

In [12]:
docs[5:6]

[Document(metadata={'source': '-', 'seq_num': 6, 'file': 'content/สรุปผลงานสภาผู้แทนราษฎร_ชุดที่_26_ปีที่_1_ครั้งที่หนึ่ง.pdf', 'page': 5}, page_content='๓ \n \n๓. ร่างพระราชบัญญัติที่อยู่ระหว่างนายกรัฐมนตรีพิจารณาให้คำรับรอง     จำนวน  ๒๓  ฉบับ \n \n \n \n๑. ร่างพระราชบัญญัติรับราชการทหาร พ.ศ. .... \nผู้เสนอ  นายพริษฐ์  วัชรสินธุ์ กับคณะ) \n๒. ร่างพระราชบัญญัติยกเลิกพระราชบัญญัติการรักษาความมั่นคงภายในราชอาณาจักร พุทธศักราช \n๒๕๕๑ พ.ศ. .... \nผู้เสนอ  นายรอมฎอน  ปันจอร์ กับคณะ  \n๓. ร่างพระราชบัญญัติกำหนดแผนและขั้นตอนการกระจายอำนาจให้แก่องค์กรปกครองส่วนท้องถิ่น  \n(ฉบับที่ ..) พ.ศ. .... \nผู้เสนอ  นายวรภพ  วิริยะโรจน์ กับคณะ \n๔. ร่างพระราชบัญญัติส่งเสริมและคุ้มครองชาติพันธุ์และชนเผ่าพื้นเมือง พ.ศ. ....  \nผู้เสนอ  นายเลาฟั้ง  บัณทิตเทอดสกุล กับคณะ  \n๕. ร่างพระราชบัญญัติการปฏิรูปที่ดินเพื่อประชาชน พ.ศ. .... \nผู้เสนอ  นายอภิชาติ  ศิริสุนทร กับคณะ  \n๖. ร่างพระราชบัญญัติคุ้มครองแรงงาน (ฉบับที่ ..) พ.ศ. .... \n(แก้ไขมาตรา ๔ และมาตรา ๔๑ กำหนดลูกจ้างให้ครอบคลุมถึงลูกจ้างส่วนราชการ และวัน

In [13]:
# Clean up
if os.path.exists(output_file_path):
    os.remove(output_file_path)

https://qdrant.tech

In [15]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from langchain_qdrant import FastEmbedSparse, RetrievalMode
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings


url="https://2ac2b7ad-e77f-4b9c-9e46-21757ab9dbdf.us-east4-0.gcp.cloud.qdrant.io:6333"
api_key = userdata.get('QDRANT_TOKEN')

client = QdrantClient(url=url,api_key=api_key)

In [16]:
client.get_collections()

CollectionsResponse(collections=[])

In [17]:
client.delete_collection(collection_name="test")

False

In [18]:
client.get_collections()

CollectionsResponse(collections=[])

In [19]:
embeddings = HuggingFaceEmbeddings(model_name='BAAI/bge-m3')
sparse_embeddings = FastEmbedSparse(model_name="Qdrant/bm25")

<ipython-input-19-e4c0dc628529>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='BAAI/bge-m3')
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

russian.txt:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

spanish.txt:   0%|          | 0.00/2.18k [00:00<?, ?B/s]

tajik.txt:   0%|          | 0.00/1.82k [00:00<?, ?B/s]

turkish.txt:   0%|          | 0.00/260 [00:00<?, ?B/s]

swedish.txt:   0%|          | 0.00/559 [00:00<?, ?B/s]

### Dense Vector Search

In [21]:
display(client.delete_collection("demo_collection"))

client.create_collection(
    collection_name="demo_collection",
    vectors_config=VectorParams(size=1024, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="demo_collection",
    embedding=embeddings,
    sparse_embedding=sparse_embeddings,
    retrieval_mode=RetrievalMode.DENSE,

)



True

In [22]:
from uuid import uuid4
uuids = [str(uuid4()) for _ in range(len(docs))]


batch_size = 128
# Process in batches
for i in trange(0, len(docs), batch_size):
    batch_docs = docs[i:i + batch_size]
    batch_ids = uuids[i:i + batch_size]
    vector_store.add_documents(documents=batch_docs, ids=batch_ids)

  0%|          | 0/1 [00:00<?, ?it/s]

In [23]:
results = vector_store.similarity_search_with_score("ใครเสนอควบคุมเครื่องดื่มแอลกอฮอล์", k=3,score_threshold=0.5)

display(results)
for doc, score in results:
    print(f"* [SIM={score:3f}] \n {doc.page_content[:200]} \n [{doc.metadata}]")

[(Document(metadata={'source': '-', 'seq_num': 10, 'file': 'content/สรุปผลงานสภาผู้แทนราษฎร_ชุดที่_26_ปีที่_1_ครั้งที่หนึ่ง.pdf', 'page': 9, '_id': '23b7244a-094e-49ef-8ca0-0eec06480181', '_collection_name': 'demo_collection'}, page_content='๗ \n \n \n- บรรจุระเบียบวาระการประชุมสภาผู้แทนราษฎร ชุดที่ ๒๖ ปีที่ ๑ ครั้งที่ ๑๕ (สมัยสามัญประจำปี\nครั้งที่หนึ่ง) วันพุธที่ ๒๐ กันยายน ๒๕๖๖ \n \n3. ร่างพระราชบัญญัติควบคุมเครื่องดื่มแอลกอฮอล์ (ฉบับที่ ..) พ.ศ. .... \nผู้เสนอ  นายธีรภัทร์  คหะวงศ์ กับผู้มีสิทธิเลือกตั้ง จำนวน ๙๒,๙๗๘ คน \nสาระ  กำหนดมาตรการควบคุมเครื่องดื่มแอลกอฮอล์ที่รัดกุมที่จะคุ้มครองประชาชน เด็ก เยาวชน ทำให้ \nรู้เท่าทันพิษภัยจากเครื่องดื่มแอลกอฮอล์ การตลาดของธุรกิจแอลกอฮอล์ มีมาตรการคุ้มครองผู้บริโภคและ\nประชาชนเพิ่มเติม \nผลการดำเนินการ  \n \n- ประธานรัฐสภาวินิจฉัยว่าเป็นร่างพระราชบัญญัติที่มีหลักการเป็นไปตามหมวด ๓ สิทธิและเสรีภาพ\nของปวงชนชาวไทย และหมวด ๕ หน้าที่ของรัฐ \n \n- ประธานสภาผู้แทนราษฎรพิจารณาวินิจฉัยว่าไม่เป็นร่างพระราชบัญญัติเกี่ยวด้วยการเงิน \n \n- สำนักงานเลขาธ

* [SIM=0.575722] 
 ๗ 
 
 
- บรรจุระเบียบวาระการประชุมสภาผู้แทนราษฎร ชุดที่ ๒๖ ปีที่ ๑ ครั้งที่ ๑๕ (สมัยสามัญประจำปี
ครั้งที่หนึ่ง) วันพุธที่ ๒๐ กันยายน ๒๕๖๖ 
 
3. ร่างพระราชบัญญัติควบคุมเครื่องดื่มแอลกอฮอล์ (ฉบับที่ ..) 
 [{'source': '-', 'seq_num': 10, 'file': 'content/สรุปผลงานสภาผู้แทนราษฎร_ชุดที่_26_ปีที่_1_ครั้งที่หนึ่ง.pdf', 'page': 9, '_id': '23b7244a-094e-49ef-8ca0-0eec06480181', '_collection_name': 'demo_collection'}]
* [SIM=0.572513] 
 ๖ 
 
การเข้าชื่อเสนอกฎหมายของประชาชนผู้มีสิทธิเลือกตั้ง 
 
๑. รอการพิจารณาของสภาในวาระที่หนึ่ง  
 
 
 
  จำนวน  ๓  ฉบับ 
 
 
 
๑. ร่างพระราชบัญญัติควบคุมเครื่องดื่มแอลกอฮอล์ (ฉบับที่ ..) พ.ศ. .... 
ผู 
 [{'source': '-', 'seq_num': 9, 'file': 'content/สรุปผลงานสภาผู้แทนราษฎร_ชุดที่_26_ปีที่_1_ครั้งที่หนึ่ง.pdf', 'page': 8, '_id': '72916019-ba54-40c8-9c14-560ae560750f', '_collection_name': 'demo_collection'}]
* [SIM=0.556097] 
 ๒ 
 
๑๑. ร่างพระราชบัญญัติควบคุมเครื่องดื่มแอลกอฮอล์ (ฉบับที่ ..) พ.ศ. .... 
ผู้เสนอ  นายเท่าพิภพ  ลิ้มจิตรกร กับคณะ 
๑๒

### Sparse Vector Search

In [26]:
client.delete_collection("demo_collection_sparse")

client.create_collection(
    collection_name="demo_collection_sparse",
    # vectors_config={
    #     "dense_vector": VectorParams(size=1024, distance=Distance.COSINE),
    # },
    sparse_vectors_config={
        "sparse_vector": {},
    },
)
vector_store = QdrantVectorStore(
    client=client,
    collection_name="demo_collection_sparse",
    sparse_embedding=sparse_embeddings,
    retrieval_mode=RetrievalMode.SPARSE,
    sparse_vector_name="sparse_vector",
)


from uuid import uuid4
uuids = [str(uuid4()) for _ in range(len(docs))]
vector_store.add_documents(documents=docs, ids=uuids)

print('finish')

TypeError: QdrantClient.create_collection() missing 1 required positional argument: 'vectors_config'

In [28]:
client.delete_collection("demo_collection_sparse")

client.create_collection(
    collection_name="demo_collection_sparse",
    vectors_config=VectorParams(size=1024, distance=Distance.COSINE), # Add vectors_config for dense vectors
    sparse_vectors_config={
        "sparse_vector": {},
    },
)
vector_store = QdrantVectorStore(
    client=client,
    collection_name="demo_collection_sparse",
    sparse_embedding=sparse_embeddings,
    retrieval_mode=RetrievalMode.SPARSE,
    sparse_vector_name="sparse_vector",
)


from uuid import uuid4
uuids = [str(uuid4()) for _ in range(len(docs))]
vector_store.add_documents(documents=docs, ids=uuids)

print('finish')

finish


In [29]:
results = vector_store.similarity_search_with_score("ใครเสนอควบคุมเครื่องดื่มแอลกอฮอล์", k=3,score_threshold=0.5)

display(results)
for doc, score in results:
    print(f"* [SIM={score:3f}] \n {doc.page_content[:200]} \n [{doc.metadata}]")

[(Document(metadata={'source': '-', 'seq_num': 10, 'file': 'content/สรุปผลงานสภาผู้แทนราษฎร_ชุดที่_26_ปีที่_1_ครั้งที่หนึ่ง.pdf', 'page': 9, '_id': '99066afd-9817-4f8c-bb42-eb44d8271415', '_collection_name': 'demo_collection_sparse'}, page_content='๗ \n \n \n- บรรจุระเบียบวาระการประชุมสภาผู้แทนราษฎร ชุดที่ ๒๖ ปีที่ ๑ ครั้งที่ ๑๕ (สมัยสามัญประจำปี\nครั้งที่หนึ่ง) วันพุธที่ ๒๐ กันยายน ๒๕๖๖ \n \n3. ร่างพระราชบัญญัติควบคุมเครื่องดื่มแอลกอฮอล์ (ฉบับที่ ..) พ.ศ. .... \nผู้เสนอ  นายธีรภัทร์  คหะวงศ์ กับผู้มีสิทธิเลือกตั้ง จำนวน ๙๒,๙๗๘ คน \nสาระ  กำหนดมาตรการควบคุมเครื่องดื่มแอลกอฮอล์ที่รัดกุมที่จะคุ้มครองประชาชน เด็ก เยาวชน ทำให้ \nรู้เท่าทันพิษภัยจากเครื่องดื่มแอลกอฮอล์ การตลาดของธุรกิจแอลกอฮอล์ มีมาตรการคุ้มครองผู้บริโภคและ\nประชาชนเพิ่มเติม \nผลการดำเนินการ  \n \n- ประธานรัฐสภาวินิจฉัยว่าเป็นร่างพระราชบัญญัติที่มีหลักการเป็นไปตามหมวด ๓ สิทธิและเสรีภาพ\nของปวงชนชาวไทย และหมวด ๕ หน้าที่ของรัฐ \n \n- ประธานสภาผู้แทนราษฎรพิจารณาวินิจฉัยว่าไม่เป็นร่างพระราชบัญญัติเกี่ยวด้วยการเงิน \n \n- สำนักง

* [SIM=4.044189] 
 ๗ 
 
 
- บรรจุระเบียบวาระการประชุมสภาผู้แทนราษฎร ชุดที่ ๒๖ ปีที่ ๑ ครั้งที่ ๑๕ (สมัยสามัญประจำปี
ครั้งที่หนึ่ง) วันพุธที่ ๒๐ กันยายน ๒๕๖๖ 
 
3. ร่างพระราชบัญญัติควบคุมเครื่องดื่มแอลกอฮอล์ (ฉบับที่ ..) 
 [{'source': '-', 'seq_num': 10, 'file': 'content/สรุปผลงานสภาผู้แทนราษฎร_ชุดที่_26_ปีที่_1_ครั้งที่หนึ่ง.pdf', 'page': 9, '_id': '99066afd-9817-4f8c-bb42-eb44d8271415', '_collection_name': 'demo_collection_sparse'}]
* [SIM=3.521467] 
 ๖ 
 
การเข้าชื่อเสนอกฎหมายของประชาชนผู้มีสิทธิเลือกตั้ง 
 
๑. รอการพิจารณาของสภาในวาระที่หนึ่ง  
 
 
 
  จำนวน  ๓  ฉบับ 
 
 
 
๑. ร่างพระราชบัญญัติควบคุมเครื่องดื่มแอลกอฮอล์ (ฉบับที่ ..) พ.ศ. .... 
ผู 
 [{'source': '-', 'seq_num': 9, 'file': 'content/สรุปผลงานสภาผู้แทนราษฎร_ชุดที่_26_ปีที่_1_ครั้งที่หนึ่ง.pdf', 'page': 8, '_id': 'c23e2266-c9ce-4a56-b6e4-337ea4561e86', '_collection_name': 'demo_collection_sparse'}]
* [SIM=2.510550] 
 ๒ 
 
๑๑. ร่างพระราชบัญญัติควบคุมเครื่องดื่มแอลกอฮอล์ (ฉบับที่ ..) พ.ศ. .... 
ผู้เสนอ  นายเท่าพิภพ  ลิ้มจิต

### Hybrid Vector Search

In [30]:
client.delete_collection("demo_collection_HYBRID")

client.create_collection(
    collection_name="demo_collection_HYBRID",
    vectors_config={
        "dense_vector": VectorParams(size=1024, distance=Distance.COSINE),
    },
    sparse_vectors_config={
        "sparse_vector": {},
    },
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="demo_collection_HYBRID",
    embedding=embeddings,
    sparse_embedding=sparse_embeddings,
    retrieval_mode=RetrievalMode.HYBRID,
    vector_name="dense_vector",
    sparse_vector_name="sparse_vector",
)


from uuid import uuid4
uuids = [str(uuid4()) for _ in range(len(docs))]


batch_size = 128
# Process in batches
for i in trange(0, len(docs), batch_size):
    batch_docs = docs[i:i + batch_size]
    batch_ids = uuids[i:i + batch_size]
    vector_store.add_documents(documents=batch_docs, ids=batch_ids)


  0%|          | 0/1 [00:00<?, ?it/s]

In [31]:
results = vector_store.similarity_search_with_score("ใครเสนอควบคุมเครื่องดื่มแอลกอฮอล์", k=3,score_threshold=0.5)

display(results)
# for doc, score in results:
#     print(f"* [SIM={score:3f}] \n {doc.page_content[:200]} \n [{doc.metadata}]")

[(Document(metadata={'source': '-', 'seq_num': 10, 'file': 'content/สรุปผลงานสภาผู้แทนราษฎร_ชุดที่_26_ปีที่_1_ครั้งที่หนึ่ง.pdf', 'page': 9, '_id': '1d0f3913-c17a-4c82-967f-2034908e9a70', '_collection_name': 'demo_collection_HYBRID'}, page_content='๗ \n \n \n- บรรจุระเบียบวาระการประชุมสภาผู้แทนราษฎร ชุดที่ ๒๖ ปีที่ ๑ ครั้งที่ ๑๕ (สมัยสามัญประจำปี\nครั้งที่หนึ่ง) วันพุธที่ ๒๐ กันยายน ๒๕๖๖ \n \n3. ร่างพระราชบัญญัติควบคุมเครื่องดื่มแอลกอฮอล์ (ฉบับที่ ..) พ.ศ. .... \nผู้เสนอ  นายธีรภัทร์  คหะวงศ์ กับผู้มีสิทธิเลือกตั้ง จำนวน ๙๒,๙๗๘ คน \nสาระ  กำหนดมาตรการควบคุมเครื่องดื่มแอลกอฮอล์ที่รัดกุมที่จะคุ้มครองประชาชน เด็ก เยาวชน ทำให้ \nรู้เท่าทันพิษภัยจากเครื่องดื่มแอลกอฮอล์ การตลาดของธุรกิจแอลกอฮอล์ มีมาตรการคุ้มครองผู้บริโภคและ\nประชาชนเพิ่มเติม \nผลการดำเนินการ  \n \n- ประธานรัฐสภาวินิจฉัยว่าเป็นร่างพระราชบัญญัติที่มีหลักการเป็นไปตามหมวด ๓ สิทธิและเสรีภาพ\nของปวงชนชาวไทย และหมวด ๕ หน้าที่ของรัฐ \n \n- ประธานสภาผู้แทนราษฎรพิจารณาวินิจฉัยว่าไม่เป็นร่างพระราชบัญญัติเกี่ยวด้วยการเงิน \n \n- สำนักง

## inference

In [ ]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from langchain_qdrant import FastEmbedSparse, RetrievalMode
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings


url="https://c4ef756d-805f-4323-9610-f958a717d059.us-east4-0.gcp.cloud.qdrant.io:6333"
api_key = userdata.get('QDRANT_TOKEN')

client = QdrantClient(url =url,api_key=api_key)

# Download model
embeddings = HuggingFaceEmbeddings(model_name='BAAI/bge-m3')
sparse_embeddings = FastEmbedSparse(model_name="Qdrant/bm25")


<ipython-input-3-b0c08bfd4f00>:14: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='BAAI/bge-m3')


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

slovene.txt:   0%|          | 0.00/16.0k [00:00<?, ?B/s]

spanish.txt:   0%|          | 0.00/2.18k [00:00<?, ?B/s]

swedish.txt:   0%|          | 0.00/559 [00:00<?, ?B/s]

turkish.txt:   0%|          | 0.00/260 [00:00<?, ?B/s]

tajik.txt:   0%|          | 0.00/1.82k [00:00<?, ?B/s]

In [32]:
vector_store = QdrantVectorStore(
    client=client,
    collection_name="demo_collection_HYBRID",
    embedding=embeddings,
    sparse_embedding=sparse_embeddings,
    retrieval_mode=RetrievalMode.HYBRID,
    vector_name="dense_vector",
    sparse_vector_name="sparse_vector",
)

In [33]:
results = vector_store.similarity_search_with_score("ใครเสนอควบคุมเครื่องดื่มแอลกอฮอล์", k=5,score_threshold=0.5)
results


[(Document(metadata={'source': '-', 'seq_num': 10, 'file': 'content/สรุปผลงานสภาผู้แทนราษฎร_ชุดที่_26_ปีที่_1_ครั้งที่หนึ่ง.pdf', 'page': 9, '_id': '1d0f3913-c17a-4c82-967f-2034908e9a70', '_collection_name': 'demo_collection_HYBRID'}, page_content='๗ \n \n \n- บรรจุระเบียบวาระการประชุมสภาผู้แทนราษฎร ชุดที่ ๒๖ ปีที่ ๑ ครั้งที่ ๑๕ (สมัยสามัญประจำปี\nครั้งที่หนึ่ง) วันพุธที่ ๒๐ กันยายน ๒๕๖๖ \n \n3. ร่างพระราชบัญญัติควบคุมเครื่องดื่มแอลกอฮอล์ (ฉบับที่ ..) พ.ศ. .... \nผู้เสนอ  นายธีรภัทร์  คหะวงศ์ กับผู้มีสิทธิเลือกตั้ง จำนวน ๙๒,๙๗๘ คน \nสาระ  กำหนดมาตรการควบคุมเครื่องดื่มแอลกอฮอล์ที่รัดกุมที่จะคุ้มครองประชาชน เด็ก เยาวชน ทำให้ \nรู้เท่าทันพิษภัยจากเครื่องดื่มแอลกอฮอล์ การตลาดของธุรกิจแอลกอฮอล์ มีมาตรการคุ้มครองผู้บริโภคและ\nประชาชนเพิ่มเติม \nผลการดำเนินการ  \n \n- ประธานรัฐสภาวินิจฉัยว่าเป็นร่างพระราชบัญญัติที่มีหลักการเป็นไปตามหมวด ๓ สิทธิและเสรีภาพ\nของปวงชนชาวไทย และหมวด ๕ หน้าที่ของรัฐ \n \n- ประธานสภาผู้แทนราษฎรพิจารณาวินิจฉัยว่าไม่เป็นร่างพระราชบัญญัติเกี่ยวด้วยการเงิน \n \n- สำนักง

In [34]:
results =[ result for result,_ in results]
results

[Document(metadata={'source': '-', 'seq_num': 10, 'file': 'content/สรุปผลงานสภาผู้แทนราษฎร_ชุดที่_26_ปีที่_1_ครั้งที่หนึ่ง.pdf', 'page': 9, '_id': '1d0f3913-c17a-4c82-967f-2034908e9a70', '_collection_name': 'demo_collection_HYBRID'}, page_content='๗ \n \n \n- บรรจุระเบียบวาระการประชุมสภาผู้แทนราษฎร ชุดที่ ๒๖ ปีที่ ๑ ครั้งที่ ๑๕ (สมัยสามัญประจำปี\nครั้งที่หนึ่ง) วันพุธที่ ๒๐ กันยายน ๒๕๖๖ \n \n3. ร่างพระราชบัญญัติควบคุมเครื่องดื่มแอลกอฮอล์ (ฉบับที่ ..) พ.ศ. .... \nผู้เสนอ  นายธีรภัทร์  คหะวงศ์ กับผู้มีสิทธิเลือกตั้ง จำนวน ๙๒,๙๗๘ คน \nสาระ  กำหนดมาตรการควบคุมเครื่องดื่มแอลกอฮอล์ที่รัดกุมที่จะคุ้มครองประชาชน เด็ก เยาวชน ทำให้ \nรู้เท่าทันพิษภัยจากเครื่องดื่มแอลกอฮอล์ การตลาดของธุรกิจแอลกอฮอล์ มีมาตรการคุ้มครองผู้บริโภคและ\nประชาชนเพิ่มเติม \nผลการดำเนินการ  \n \n- ประธานรัฐสภาวินิจฉัยว่าเป็นร่างพระราชบัญญัติที่มีหลักการเป็นไปตามหมวด ๓ สิทธิและเสรีภาพ\nของปวงชนชาวไทย และหมวด ๕ หน้าที่ของรัฐ \n \n- ประธานสภาผู้แทนราษฎรพิจารณาวินิจฉัยว่าไม่เป็นร่างพระราชบัญญัติเกี่ยวด้วยการเงิน \n \n- สำนักงา

In [36]:
from google.colab import userdata
import os
from groq import Groq

class GroqCall():

    def __init__(self,
                api= userdata.get('GROQ_API'),
                model_name= "meta-llama/llama-4-maverick-17b-128e-instruct",
                ):

        self.client = Groq(api_key=userdata.get('GROQ_API'))
        self.model_name = model_name

    def generate(self, messages, system, **kwargs):
        # Add the system prompt to the messages
        messages = [
            {"role": "system", "content": system},
            *messages,
        ]

        response = self.client.chat.completions.create(
            model=self.model_name,
            messages=messages,
            max_tokens=4096,
            temperature=0.1,
            stream=False,
            extra_body=kwargs
        )

        return response.choices[0].message.content



In [37]:
system='''\
You are a highly intelligent AI assistant. You will be given a context, and based on that context,\
you need to answer questions accurately. Please ensure your answers are based only on the provided context.\
Do not include information that is not present in the context. Provide concise and clear responses.\
Additionally, include references and page numbers in your answers where applicable.

Instructions:
    •	Read the context carefully.
    •	Answer each question based solely on the information provided in the context.
    •	Ensure your answers are precise and relevant to the questions.
    •	Include references and page numbers in your answers where applicable.
    •	Correct Thai spell word.\
'''

In [38]:
def _get_meassage(Questions,Context):

    user_prompt  =  [{"role": "user", "content": f'''\
Context:

{Context}

Questions:

{Questions}'''}]
    return user_prompt

In [39]:
query = 'ใครเสนอควบคุมเครื่องดื่มแอลกอฮอล์'

_get_meassage(query,str(results))

[{'role': 'user',
  'content': "Context:\n\n[Document(metadata={'source': '-', 'seq_num': 10, 'file': 'content/สรุปผลงานสภาผู้แทนราษฎร_ชุดที่_26_ปีที่_1_ครั้งที่หนึ่ง.pdf', 'page': 9, '_id': '1d0f3913-c17a-4c82-967f-2034908e9a70', '_collection_name': 'demo_collection_HYBRID'}, page_content='๗ \\n \\n \\n- บรรจุระเบียบวาระการประชุมสภาผู้แทนราษฎร ชุดที่ ๒๖ ปีที่ ๑ ครั้งที่ ๑๕ (สมัยสามัญประจำปี\\nครั้งที่หนึ่ง) วันพุธที่ ๒๐ กันยายน ๒๕๖๖ \\n \\n3. ร่างพระราชบัญญัติควบคุมเครื่องดื่มแอลกอฮอล์ (ฉบับที่ ..) พ.ศ. .... \\nผู้เสนอ  นายธีรภัทร์  คหะวงศ์ กับผู้มีสิทธิเลือกตั้ง จำนวน ๙๒,๙๗๘ คน \\nสาระ  กำหนดมาตรการควบคุมเครื่องดื่มแอลกอฮอล์ที่รัดกุมที่จะคุ้มครองประชาชน เด็ก เยาวชน ทำให้ \\nรู้เท่าทันพิษภัยจากเครื่องดื่มแอลกอฮอล์ การตลาดของธุรกิจแอลกอฮอล์ มีมาตรการคุ้มครองผู้บริโภคและ\\nประชาชนเพิ่มเติม \\nผลการดำเนินการ  \\n \\n- ประธานรัฐสภาวินิจฉัยว่าเป็นร่างพระราชบัญญัติที่มีหลักการเป็นไปตามหมวด ๓ สิทธิและเสรีภาพ\\nของปวงชนชาวไทย และหมวด ๕ หน้าที่ของรัฐ \\n \\n- ประธานสภาผู้แทนราษฎรพิจารณาวินิจฉั

In [40]:
model = GroqCall()

In [41]:
response = model.generate(_get_meassage(query,str(results)),system)
print(response)

ร่างพระราชบัญญัติควบคุมเครื่องดื่มแอลกอฮอล์ (ฉบับที่ ..) พ.ศ. .... มีผู้เสนอ 2 คน คือ 

1. นายเจริญ เจริญชัย กับผู้มีสิทธิเลือกตั้ง จำนวน 10,942 คน (หน้า 8)
2. นายธีรภัทร์ คหะวงศ์ กับผู้มีสิทธิเลือกตั้ง จำนวน 92,978 คน (หน้า 9)
และ นายเท่าพิภพ ลิ้มจิตรกร กับคณะ (หน้า 4)


In [42]:
query = "ผลของ ร่างพระราชบัญญัติกัญชา กัญชง แห่งชาติ (หน้า10) เป็นอย่างไร สรุปเป็น bullet"
results = vector_store.similarity_search_with_score(query, k=5,score_threshold=0.5)
print(results[0][1])
results =[ result for result,_ in results]
response = model.generate(_get_meassage(query,str(results)),system)
print(response)

1.0
ผลของ ร่างพระราชบัญญัติกัญชา กัญชง แห่งชาติ พ.ศ. .... (หน้า 10) มีดังนี้
* ประธานสภาผู้แทนราษฎรวินิจฉัยว่าเป็นร่างพระราชบัญญัติที่มีหลักการเป็นไปตามหมวด 3 สิทธิและเสรีภาพของปวงชนชาวไทย
* อยู่ระหว่างการเชิญชวนให้มีการเข้าชื่อเสนอกฎหมาย (หน้า 10)


In [43]:
query = "สรุป ร่างพระราชบัญญัติทั้งหมดมีอะไรบ้าง"
results = vector_store.similarity_search_with_score(query, k=5,score_threshold=0.5)
print(results[0][1])
results =[ result for result,_ in results]
response = model.generate(_get_meassage(query,str(results)),system)
print(response)

0.5
จากเอกสารที่ให้มา มีการกล่าวถึงร่างพระราชบัญญัติหลายฉบับ โดยมีรายละเอียดดังนี้

1. ร่างพระราชบัญญัติควบคุมเครื่องดื่มแอลกอฮอล์ (ฉบับที่ ..) พ.ศ. ....
   - ผู้เสนอ: นายธีรภัทร์ คหะวงศ์ กับผู้มีสิทธิเลือกตั้ง จำนวน 92,978 คน
   - สาระ: กำหนดมาตรการควบคุมเครื่องดื่มแอลกอฮอล์ที่รัดกุมเพื่อคุ้มครองประชาชน เด็ก และเยาวชน

2. ร่างพระราชบัญญัติคุ้มครองและส่งเสริมภูมิปัญญาการแพทย์แผนไทย (ฉบับที่ ..) พ.ศ. ....
   - ผู้เสนอ: นายสุรสีห์ ลานนท์ กับคณะผู้เชิญชวน
   - สาระ: กำหนดให้จัดตั้งกระทรวงการแพทย์แผนไทยและการแพทย์ทางเลือก

นอกจากนี้ยังมีร่างพระราชบัญญัติอื่นๆ ที่อยู่ระหว่างการพิจารณาให้คำรับรองจากนายกรัฐมนตรี ได้แก่
1. ร่างพระราชบัญญัติรับราชการทหาร พ.ศ. ....
2. ร่างพระราชบัญญัติยกเลิกพระราชบัญญัติการรักษาความมั่นคงภายในราชอาณาจักร พุทธศักราช 2551 พ.ศ. ....
3. ร่างพระราชบัญญัติกำหนดแผนและขั้นตอนการกระจายอำนาจให้แก่องค์กรปกครองส่วนท้องถิ่น (ฉบับที่ ..) พ.ศ. ....
4. ร่างพระราชบัญญัติส่งเสริมและคุ้มครองชาติพันธุ์และชนเผ่าพื้นเมือง พ.ศ. ....
5. ร่างพระราชบัญญัติการปฏิรูปที่ดินเพื่อประชาชน พ.ศ.

In [44]:
# prompt: Create Function to using RAG
# input is query
# output is response from LLM

def rag_query(query):
    """
    Performs a query using RAG (Retrieval Augmented Generation).

    Args:
        query: The user's query.

    Returns:
        The response from the LLM.
    """
    results = vector_store.similarity_search_with_score(query, k=5, score_threshold=0.5)
    results = [result for result, _ in results]
    response = model.generate(_get_meassage(query, str(results)), system)
    return response , results

In [45]:
query = "สรุป พระราชบัญญัติควบคุมเครื่องดื่มแอลกอฮอล์ เป็นอย่างไร สรุปเป็น bullet และใครเป็นผู้เสนอ"
output,doc  = rag_query(query)

print(output)

print('===================')
print('===================')
print('===================')
display(doc)

พระราชบัญญัติควบคุมเครื่องดื่มแอลกอฮอล์ มีรายละเอียดดังนี้

*   ร่างพระราชบัญญัติควบคุมเครื่องดื่มแอลกอฮอล์ (ฉบับที่ ..) พ.ศ. ....
*   ผู้เสนอ: 
    1.  นายเจริญ เจริญชัย กับผู้มีสิทธิเลือกตั้ง จำนวน 10,942 คน (หน้า 8)
    2.  นายธีรภัทร์ คหะวงศ์ กับผู้มีสิทธิเลือกตั้ง จำนวน 92,978 คน (หน้า 9)
    3.  นายเท่าพิภพ ลิ้มจิตรกร กับคณะ (หน้า 4)

โดยทั้ง 3 ร่าง มีสาระสำคัญเกี่ยวกับการกำหนดมาตรการควบคุมเครื่องดื่มแอลกอฮอล์ที่รัดกุมเพื่อคุ้มครองประชาชน เด็ก และเยาวชน ให้รู้เท่าทันพิษภัยจากเครื่องดื่มแอลกอฮอล์ และการตลาดของธุรกิจแอลกอฮอล์ โดยมีมาตรการคุ้มครองผู้บริโภคและประชาชนเพิ่มเติม (หน้า 8-9) 

อ้างอิง: หน้า 8-9 และ 4


[Document(metadata={'source': '-', 'seq_num': 9, 'file': 'content/สรุปผลงานสภาผู้แทนราษฎร_ชุดที่_26_ปีที่_1_ครั้งที่หนึ่ง.pdf', 'page': 8, '_id': 'ec8dbb95-de3d-4a4f-a19a-f0f3dec331bc', '_collection_name': 'demo_collection_HYBRID'}, page_content='๖ \n \nการเข้าชื่อเสนอกฎหมายของประชาชนผู้มีสิทธิเลือกตั้ง \n \n๑. รอการพิจารณาของสภาในวาระที่หนึ่ง  \n \n \n \n  จำนวน  ๓  ฉบับ \n \n \n \n๑. ร่างพระราชบัญญัติควบคุมเครื่องดื่มแอลกอฮอล์ (ฉบับที่ ..) พ.ศ. .... \nผู้เสนอ  นายเจริญ  เจริญชัย กับผู้มีสิทธิเลือกตั้ง จำนวน ๑๐,๙๔๒ คน \nสาระ  พระราชบัญญัติควบคุมเครื่องดื่มแอลกอฮอล์ พ.ศ. ๒๕๕๑ มีการจำกัดสิทธิและเสรีภาพของบุคคล \nเกินสมควรทำให้บุคคลไม่สามารถประกอบอาชีพสุจริตได้อย่างอิสระ เป็นหลักเกณฑ์ที่ไม่คุ้มครองบุคคลให้มี\nเสรีภาพในการประกอบอาชีพอย่างเหมาะสม เพียงพอตามที่รัฐธรรมนูญแห่งราชอาณาจักรไทย มาตรา ๔๐  \nได้กำหนดไว้   \nผลการดำเนินการ  \n \n- ประธานรัฐสภาวินิจฉัยว่าเป็นร่างพระราชบัญญัติที่มีหลักการเป็นไปตามหมวด ๓ สิทธิและเสรีภาพ\nของปวงชนชาวไทย \n \n- ประธานสภาผู้แทนราษฎรพิจารณาวินิจฉัยว่าไม่เป

### Rerank

In [46]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [47]:
retriever = vector_store.as_retriever(search_type="similarity_score_threshold",
            search_kwargs={
                "k": 50,
                'score_threshold': 0.2
                })


In [48]:
query = "ใครเสนอควบคุมเครื่องดื่มแอลกอฮอล์"
docs = retriever.invoke(query)
pretty_print_docs(docs)

Document 1:

๗ 
 
 
- บรรจุระเบียบวาระการประชุมสภาผู้แทนราษฎร ชุดที่ ๒๖ ปีที่ ๑ ครั้งที่ ๑๕ (สมัยสามัญประจำปี
ครั้งที่หนึ่ง) วันพุธที่ ๒๐ กันยายน ๒๕๖๖ 
 
3. ร่างพระราชบัญญัติควบคุมเครื่องดื่มแอลกอฮอล์ (ฉบับที่ ..) พ.ศ. .... 
ผู้เสนอ  นายธีรภัทร์  คหะวงศ์ กับผู้มีสิทธิเลือกตั้ง จำนวน ๙๒,๙๗๘ คน 
สาระ  กำหนดมาตรการควบคุมเครื่องดื่มแอลกอฮอล์ที่รัดกุมที่จะคุ้มครองประชาชน เด็ก เยาวชน ทำให้ 
รู้เท่าทันพิษภัยจากเครื่องดื่มแอลกอฮอล์ การตลาดของธุรกิจแอลกอฮอล์ มีมาตรการคุ้มครองผู้บริโภคและ
ประชาชนเพิ่มเติม 
ผลการดำเนินการ  
 
- ประธานรัฐสภาวินิจฉัยว่าเป็นร่างพระราชบัญญัติที่มีหลักการเป็นไปตามหมวด ๓ สิทธิและเสรีภาพ
ของปวงชนชาวไทย และหมวด ๕ หน้าที่ของรัฐ 
 
- ประธานสภาผู้แทนราษฎรพิจารณาวินิจฉัยว่าไม่เป็นร่างพระราชบัญญัติเกี่ยวด้วยการเงิน 
 
- สำนักงานเลขาธิการสภาผู้แทนราษฎร จัดให้มีการรับฟังความคิดเห็นตามรัฐธรรมนูญ 
แห่งราชอาณาจักรไทย มาตรา ๗๗ 
 
- บรรจุระเบียบวาระการประชุมสภาผู้แทนราษฎร ชุดที่ ๒๕ ปีที่ ๔ครั้งที่ ๒๐ (สมัยสามัญประจำปี 
ครั้งที่หนึ่ง) วันพุธที่ ๒๔ สิงหาคม พ.ศ. ๒๕๖๕ 
 
- ผู้แทนของผู้เ

In [50]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")


In [51]:
compressor = CrossEncoderReranker(model=model, top_n=3)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(query)
pretty_print_docs(compressed_docs)

Document 1:

๖ 
 
การเข้าชื่อเสนอกฎหมายของประชาชนผู้มีสิทธิเลือกตั้ง 
 
๑. รอการพิจารณาของสภาในวาระที่หนึ่ง  
 
 
 
  จำนวน  ๓  ฉบับ 
 
 
 
๑. ร่างพระราชบัญญัติควบคุมเครื่องดื่มแอลกอฮอล์ (ฉบับที่ ..) พ.ศ. .... 
ผู้เสนอ  นายเจริญ  เจริญชัย กับผู้มีสิทธิเลือกตั้ง จำนวน ๑๐,๙๔๒ คน 
สาระ  พระราชบัญญัติควบคุมเครื่องดื่มแอลกอฮอล์ พ.ศ. ๒๕๕๑ มีการจำกัดสิทธิและเสรีภาพของบุคคล 
เกินสมควรทำให้บุคคลไม่สามารถประกอบอาชีพสุจริตได้อย่างอิสระ เป็นหลักเกณฑ์ที่ไม่คุ้มครองบุคคลให้มี
เสรีภาพในการประกอบอาชีพอย่างเหมาะสม เพียงพอตามที่รัฐธรรมนูญแห่งราชอาณาจักรไทย มาตรา ๔๐  
ได้กำหนดไว้   
ผลการดำเนินการ  
 
- ประธานรัฐสภาวินิจฉัยว่าเป็นร่างพระราชบัญญัติที่มีหลักการเป็นไปตามหมวด ๓ สิทธิและเสรีภาพ
ของปวงชนชาวไทย 
 
- ประธานสภาผู้แทนราษฎรพิจารณาวินิจฉัยว่าไม่เป็นร่างพระราชบัญญัติเกี่ยวด้วยการเงิน 
 
- สำนักงานเลขาธิการสภาผู้แทนราษฎร จัดให้มีการรับฟังความคิดเห็นตามรัฐธรรมนูญ 
แห่งราชอาณาจักรไทย มาตรา ๗๗ 
 
- บรรจุระเบียบวาระการประชุมสภาผู้แทนราษฎร ชุดที่ ๒๕ ปีที่ ๓ ครั้งที่ ๔ (สมัยสามัญประจำปี 
ครั้งที่สอง) วันพุธ

In [52]:
retriever = vector_store.as_retriever(search_type="similarity_score_threshold",
            search_kwargs={
                "k": 5,
                'score_threshold': 0.2
                })
compressor = CrossEncoderReranker(model=model, top_n=1)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)
query = "ใครเสนอควบคุมเครื่องดื่มแอลกอฮอล์"

compressed_docs = compression_retriever.invoke(query)
pretty_print_docs(compressed_docs)

Document 1:

๖ 
 
การเข้าชื่อเสนอกฎหมายของประชาชนผู้มีสิทธิเลือกตั้ง 
 
๑. รอการพิจารณาของสภาในวาระที่หนึ่ง  
 
 
 
  จำนวน  ๓  ฉบับ 
 
 
 
๑. ร่างพระราชบัญญัติควบคุมเครื่องดื่มแอลกอฮอล์ (ฉบับที่ ..) พ.ศ. .... 
ผู้เสนอ  นายเจริญ  เจริญชัย กับผู้มีสิทธิเลือกตั้ง จำนวน ๑๐,๙๔๒ คน 
สาระ  พระราชบัญญัติควบคุมเครื่องดื่มแอลกอฮอล์ พ.ศ. ๒๕๕๑ มีการจำกัดสิทธิและเสรีภาพของบุคคล 
เกินสมควรทำให้บุคคลไม่สามารถประกอบอาชีพสุจริตได้อย่างอิสระ เป็นหลักเกณฑ์ที่ไม่คุ้มครองบุคคลให้มี
เสรีภาพในการประกอบอาชีพอย่างเหมาะสม เพียงพอตามที่รัฐธรรมนูญแห่งราชอาณาจักรไทย มาตรา ๔๐  
ได้กำหนดไว้   
ผลการดำเนินการ  
 
- ประธานรัฐสภาวินิจฉัยว่าเป็นร่างพระราชบัญญัติที่มีหลักการเป็นไปตามหมวด ๓ สิทธิและเสรีภาพ
ของปวงชนชาวไทย 
 
- ประธานสภาผู้แทนราษฎรพิจารณาวินิจฉัยว่าไม่เป็นร่างพระราชบัญญัติเกี่ยวด้วยการเงิน 
 
- สำนักงานเลขาธิการสภาผู้แทนราษฎร จัดให้มีการรับฟังความคิดเห็นตามรัฐธรรมนูญ 
แห่งราชอาณาจักรไทย มาตรา ๗๗ 
 
- บรรจุระเบียบวาระการประชุมสภาผู้แทนราษฎร ชุดที่ ๒๕ ปีที่ ๓ ครั้งที่ ๔ (สมัยสามัญประจำปี 
ครั้งที่สอง) วันพุธ